In [3]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure

In [4]:
df = pd.read_csv('datasets/alldat.csv')

In [5]:
import os
os.listdir("./")

['.ipynb_checkpoints',
 'data exploration.ipynb',
 'datasets',
 'Price prediction.ipynb',
 'Untitled.ipynb']

In [6]:
dat = np.array(df)[:,1:]
dat.shape

(97890, 42)

In [7]:
print(dat[0,])

[2.248500e+02 1.162400e+02 6.262000e+01 7.991200e+02 1.157300e+02
 7.767800e+02 3.656500e+01 8.559000e+01 5.402000e+01 1.169545e+03
 2.255000e+01 1.765400e+02 8.720000e+01 9.079000e+01 2.954000e+01
 1.618000e+02 7.890000e+01 1.141900e+02 5.581000e+01 1.184000e+02
 1.352000e+02 2.544000e+03 1.390000e+03 4.120000e+02 3.000000e+02
 1.479000e+04 1.000000e+02 1.000000e+00 0.000000e+00 1.300000e+03
 0.000000e+00 2.946300e+04 0.000000e+00 3.150000e+03 1.000000e+02
 0.000000e+00 4.300000e+01 1.000000e+02 0.000000e+00 2.510000e+03
 2.000000e+00 4.000000e+02]


In [8]:
nMin = 390
nDay = int(dat.shape[0]/nMin)
print(nMin)
print(nDay)

window = 21

390
251


In [9]:
def createCNNFeatureVectors(dat, window):
    vec_list = []
    for day in range(nDay):
        base = day * nMin
        for min_ind in range(window, nDay):
            vec = np.zeros((2, 21, window))
            vec[0,:,:] = dat[(base+min_ind-window):(base+min_ind), :21].T
            vec[1,:,:] = dat[(base+min_ind-window):(base+min_ind), 21:].T
            vec_list.append(vec)
    return np.array(vec_list)
            

In [25]:
def createClassLabels(dat, window, yCol):
    labels = np.array(nDay * (nMin - window))
    for day in range(nDay):
        base = day * nMin
        for min_ind in range(window, nDay):
            labels[day * (nMin - window) : (day + 1) * (nMin - window)] = dat[(base+min_ind-window+1):(base+min_ind+1), yCol] / dat[(base+min_ind-window):(base+min_ind), yCol] - 1

In [10]:
X_dat = createCNNFeatureVectors(dat, 20)
X_dat.nbytes

389632320

In [11]:
changes = dat[1:] - dat[:-1]

In [20]:
mean_changes = np.zeros(21)
sd_changes = np.zeros(21)
for i in range(21):
    mean_changes[i] = np.mean(changes[:,i])
    sd_changes[i] = np.std(changes[:,i])
print(mean_changes)
print(sd_changes)

[ 4.29363871e-04  5.41838204e-04  2.34653536e-04  2.59794257e-03
  6.21213824e-04  2.75434421e-03  9.82745763e-05  1.10733075e-02
 -1.01134959e-05 -9.91853017e-03  7.14074104e-05 -3.77161887e-04
  2.03802266e-04 -7.14074104e-05  6.84959495e-05  6.01293302e-04
  3.60510374e-04 -7.69289706e-04  5.01588534e-05  7.15095670e-05
  5.56446587e-04]
[0.05510104 0.08809776 0.0377387  0.50475875 0.08917401 0.485472
 0.02213673 2.22870084 0.02760194 3.21386199 0.01818523 0.20117358
 0.0517931  0.03124918 0.01785081 0.09380702 0.04193038 0.2291431
 0.03371308 0.05280791 0.07056027]


In [22]:
returns = (dat[1:,:21] / dat[:-1,:21])-1

In [24]:
mean_returns = np.zeros(21)
sd_returns = np.zeros(21)
for i in range(21):
    mean_returns[i] = np.mean(returns[:,i])
    sd_returns[i] = np.std(returns[:,i])
print(mean_returns)
print(sd_returns)

[ 1.77622143e-06  4.00779920e-06  3.32163678e-06  2.96299135e-06
  4.46974924e-06  3.17859242e-06  2.54530023e-06  8.45311531e-05
 -2.67727506e-08 -6.76874603e-06  3.03345628e-06 -1.52323369e-06
  2.26229344e-06 -7.45891947e-07  2.23762397e-06  3.29327838e-06
  3.86814190e-06 -7.22884559e-06  1.04866126e-06  6.98997066e-07
  3.55883197e-06]
[0.00022634 0.00058119 0.00051107 0.00053046 0.00055909 0.00051972
 0.00056525 0.0249915  0.00056982 0.00277245 0.00074195 0.00116914
 0.00056187 0.00038084 0.00055058 0.00049758 0.00042836 0.00206378
 0.00061191 0.00047332 0.00044859]


In [ ]:
import torch
from torch import nn


In [ ]:
class priceNet(nn.Module):
    def __init__(self):
        self.conv1 